# First Installation Guide:

For using the ssc-cdi interfaces for the first time, you need to install the ssc-cdi package on the machine of interest. To do that, following the following steps:

**1. Open up the terminal and log in to both the machines of interest using ssh:**
- ssh user.name@bertha.lnls.br
- ssh user.name@mafalda.lnls.br
    
**2. Go to the folder containg the installation scripts using the following command:**
- cd /ibira/lnls/beamlines/caterete/apps/jupyter/

**3. Run the scripts at each of the respective machines:**
- bash install_bertha.bash
- bash install_mafalda.bash

If succesful, you can now run the interface as explained below.

---

# User Guide for ssc-cdi Jupyter Interfaces:

To access JupyterLab, open the following address in the browser: [http://10.31.31.16/](http://10.31.31.16/).

This interface runs at Bertha. There a few templates that can be accessed  in Template > gcc > /gcc/template_name.ipynb.


## 1. Unwrap Interface:

Under construction.

---


## 2. Ptychography Interface:

Running the first Jupyter cell will prompt you to connect using your credentials and then install all the required packages for running the interface.

Under construction.

---


## 3. Tomography Interface:

Running the first Jupyter cell will prompt you to connect using your credentials and then install all the required packages for running the interface.

Then, the second cell will deploy the graphical interface.

Currently, a 3D reconstruction requires the following processing steps, each corresponding to a different tab of the interface:

#### **1. Selecting data and sorting it by rotation angle**

Here you will need to indicate the data folders of interest for the tomographic reconstruction. These are:
 - Ibira Datapath: the folder of the proposal containing the acquisition folder. 
 - Ibira Datafolders: a list containing the folder of the acquisition inside "Ibira Datapath" (or the multiple acquisitions for a single sample, when that is the case)
 - Ptycho sinogram path: full path to the .npy sinogram containing all frames, i.e. the output of the ptychography algorithm. This folder should be according to the standard defined by Caterete: 00000000/proc/recons/myDataFolder/
 
IMPORTANT: all outputs are saved in the same folder of the Ptychography sinogram above!
 
After selecting the correct folders, click the "Sort Frames" button. After frame are sorted, the plot should appear on the right (LOADING MIGHT TAKE A WHILE!) and can be controlled with the slider bar / play button.

#### **2. Cropping undesired borders of the dataset**

This is required to perform adequate phase unwrapping, since the noisy border interfere with the unwrapping algorithm

Click the "Load Frames" button to load the sorted data from the previous tab. Then, use the sliders to crop as wanted in each direction. Make sure to visualize if the cropping is adequate for all frames. After chosing, click "Save cropped frames" to save the cropped sinogram.

#### **3. Phase Unwrapping**

First, load the cropped sinogram with the "Load cropped frames" button. Then, find all the frames that are bad and list them in the "Bad frames" field as a list of numbers in square brackets. You can play the video of all frames as slow as wanted by changing the time each frime is visible in the "Time/frame" box. After listing all bad frames, click "Remove Bad frames".

To perform phase unwrapping, you can set the number of Unwrap Iterations (standard = 0), which will iteratively try to remove background gradient in the images. The Non-negativity checkbox will make all values positive after the iterations; the Gradient checkbox will to a final gradient removal after the non-negativity step. Both of these should be false, a priori.

After that, click "Perform Unwrap" to start unwrapping. You can view the progress in the Log Console (View > Show Log Console). After unwrapping is done, the unwrapped frames should be visible on the right. Click "Save unwrapped frames" to save the unwrapped sinogram for the next step.

#### **4. Convex Hull**
    
This step is OPTIONAL. It consists in an approach to select region around the object and null any existing null outside thie region. This should be used only when such noise is significant. First, load the unwrapped sinogram. The "Convex Hull Preview" will apply Convex Hull only to the frame that is currently selected, so you can quickly adjust the parameters (Invert, Threshold, Opening, Erosing, ConvexHull) and visualize its results. If no region seems to be found, the first step is to check/uncheck the "Invert" box. The remaining parameters need to be adjusted empirically to find the best convex region around the sample (which will be shown in the plot entitled "Convex Hull"). The plot with name Masked Image is the product of the Convex Hull mask with the original image, and it will be the frame saved to the output sinogram.

After you have found a satisfactory combination of parameters, click "Do complete Convex Hull" to do it for all frames. After this is complete, click "Load cHull sinogram" to visualize all the frames with Convex Hull and check if any were bad. If so, these can be removed by inserting their number in the Bad Frames fields as a list in squared brackets, and clicking "Remove Bad Frames"
    
    
#### **5. Wiggle**

This step consists in the alignment of the shifted frames of the sinogram, with respect to a reference frame. This is the most time consuming part of the processing for now.

You can either load the sinogram from Phase Unwrapping or Convex Hull. The loadad frames should be used to find a Reference Frame with respect to which alignment will be performed. We recommend this frame to have the sample as centered as possible. Choose the number of CPUs to use in this part, which is performed in parallel CPUs. We recommends at least 32.

Click "Perform Wiggle". After it is done, click "Load Wiggle" to show wiggled frames on the right-most plot.

You can visualize the different slices of the sinogram in the XY and XZ directions with the Sinogram sliders. 

IMPORTANT: the "Invert Sinogram" button is used to overwrite the wiggled sinogram with the opposite sine (i.e. times -1). This should be done when the sinogram presents negative values for the sample (dark) and positive (bright) for the background.


#### **6. Tomography** 
 
 - Currently, we can only use the tomography EEM algorithm. We reccomend trying it without regulirization at first.
 - The regularization parameter can be played with to try and improve the reconstruction quality. 
 - The number of iterations will not necessarily improve reconstruction quality as it gets bigger. In fact, too many iterations might deteriorate quality for some samples. We recommend values between 20 and 40 to start with. 
 - GPUs list: should be a list containing the indexes for each GPU to be used. For instance [0,1,2,3] if you want to use 4 GPUs; [0,1] if you want use 2 GPUs. [0,1,2] if you want 3 GPUs.
 - N of CPUS: The recommendation is to select 32 CPUs for each GPU used.
 - Machine Queue: this is the cluster queue to be used when you are running this with via Mafalda. Ignore it if you are using Bertha.
 - Slurm Job Name: the name of the job that will be sent to the cluster when you run it via Mafalda.
 
 After adjusting these parametes, select which steps of the tomographic processing you want to perform. If you already created the sinograms in the previous steps, you may select only the "Tomo" checkbox. Otherwise, you can select the one you want to re-run any steps, as wanted. Click "Save JSON" to save the file with the input parameters for the tomography. Then click "Start". 
 
 Check the LogConsole to see when the tomography finishes. Then, click "Load Recon Slices" to visualize the frames in each direction. These can be controlled using the sliders. In some cases, the reconstruction present outlier values that do not allow one to properly visualize anything in the plots. In that case you can set a threshold value and save a thresholded version of the 3D reconstruction. For instance, if you set it to 100, any voxel presenting value above 100 or below -100 will be set to zero. 
 
All reconsutructions are saved both in numpy and tif format!

After all steps are done, you can click the "Delete temporaty files" button to erase all intermediary sinograms that were generated, that is, you will be left only with the original Ptychography sinogram and the 3D reconstruction files.